In [176]:
from datetime import  timedelta
import pandas as pd
import os
import json

In [62]:
print(os.getcwd())
path = os.getcwd() + '/source/'

/Users/nicolascorchuelo/Documents/projects/meli


In [302]:
# Abre el archivo y carga los datos
with open(path + 'prints.json', 'r') as f:
    data = [json.loads(line) for line in f]

df_prints = pd.DataFrame(data)

df_prints = df_prints.join(df_prints['event_data'].apply(pd.Series))
df_prints = df_prints.rename(columns=lambda x: x + '_prints')
df_prints['day_prints'] = pd.to_datetime(df_prints['day_prints'])
df_prints = df_prints.drop(['event_data_prints'], axis=1)

In [184]:
now = df_prints['day_prints'].max()
week_ago = now - timedelta(days=7)
three_weeks_ago = now - timedelta(weeks=3)

In [189]:

df_last_week = df_prints[(df_prints['day_prints'] >= week_ago) & (df_prints['day_prints'] <= now)]
df_last_week = df_last_week.rename(columns=lambda x: x + '_last_week')

In [210]:

filtered_df_last_week_users_prop_prints = pd.merge(df_prints, df_last_week, how='inner', left_on=['user_id_prints', 'value_prop_prints'], right_on=['user_id_prints_last_week', 'value_prop_prints_last_week'])
df_last_three_weeks = filtered_df_last_week_users_prop_prints[(filtered_df_last_week_users_prop_prints['day_prints'] >= three_weeks_ago) & (filtered_df_last_week_users_prop_prints['day_prints'] <= now)]
counts_prints = df_last_three_weeks.groupby(['user_id_prints', 'value_prop_prints']).size()
counts_df_prints = counts_prints.reset_index(name='counts_prints')

In [185]:
# Abre el archivo y carga los datos
with open(path + 'taps.json', 'r') as f:
    data = [json.loads(line) for line in f]

df_taps = pd.DataFrame(data)
df_taps = df_taps.join(df_taps['event_data'].apply(pd.Series))
df_taps = df_taps.rename(columns=lambda x: x + '_taps')
df_taps['day_taps'] = pd.to_datetime(df_taps['day_taps'])
df_taps = df_taps.drop(['event_data_taps'], axis=1)
df_taps

,day_taps,user_id_taps,position_taps,value_prop_taps
0,2020-11-01,98702,0,cellphone_recharge
1,2020-11-01,3708,2,point
2,2020-11-01,3708,3,send_money
3,2020-11-01,93963,0,transport
4,2020-11-01,93963,1,cellphone_recharge
...,...,...,...,...
50854,2020-11-30,64149,2,credits_consumer
50855,2020-11-30,3329,1,point
50856,2020-11-30,77012,1,cellphone_recharge
50857,2020-11-30,14704,2,cellphone_recharge


In [211]:
merged_df_taps_last_week = pd.merge(df_taps, df_last_week, how='inner', left_on=['user_id_taps', 'value_prop_taps'], right_on=['user_id_prints_last_week', 'value_prop_prints_last_week'])
df1_last_three_weeks = merged_df_taps_last_week[(merged_df_taps_last_week['day_taps'] >= three_weeks_ago) & (merged_df_taps_last_week['day_taps'] <= now)]
counts_taps = df1_last_three_weeks.groupby(['user_id_taps', 'value_prop_taps']).size()
counts_df_taps = counts_taps.reset_index(name='counts_taps')

In [206]:
q1 = pd.merge(df_last_week, df_taps, how='inner', left_on=['day_prints_last_week', 'user_id_prints_last_week','value_prop_prints_last_week'], right_on=['day_taps', 'user_id_taps','value_prop_taps'])
counts_q1 = q1.groupby(['day_prints_last_week', 'user_id_prints_last_week','value_prop_prints_last_week']).size()
counts_q1_prints = counts_q1.reset_index(name='counts_q1')

In [235]:
merged_q1 = pd.merge(df_last_week, counts_q1_prints, how='left', left_on=['day_prints_last_week', 'user_id_prints_last_week','value_prop_prints_last_week'], right_on=['day_prints_last_week', 'user_id_prints_last_week','value_prop_prints_last_week'])
merged_q1['counts_q1'] = merged_q1['counts_q1'].fillna(0)
merged_q1['clicked_or_not'] = merged_q1['counts_q1'].apply(lambda x: True if x == 1 else False)

In [275]:
merged_q2 = pd.merge(merged_q1, counts_df_prints, how='left', left_on=['user_id_prints_last_week', 'value_prop_prints_last_week'], right_on=['user_id_prints', 'value_prop_prints'])
merged_q2['count_number_prop_views_3_weeks'] = merged_q2['counts_prints'].fillna(0)


In [285]:
merged_q3 = pd.merge(merged_q2, counts_df_taps, how='left', left_on=['user_id_prints_last_week', 'value_prop_prints_last_week'], right_on=['user_id_taps', 'value_prop_taps'])
merged_q3['count_number_prop_click_3_weeks'] = merged_q3['counts_taps'].fillna(0)

In [199]:
df_pays = pd.read_csv(path + 'pays.csv')
df_pays = df_pays.rename(columns=lambda x: x + '_pays')
df_pays['pay_date_pays'] = pd.to_datetime(df_pays['pay_date_pays'])


,pay_date_pays,total_pays,user_id_pays,value_prop_pays
0,2020-11-01,7.04,35994,link_cobro
1,2020-11-01,37.36,79066,cellphone_recharge
2,2020-11-01,15.84,19321,cellphone_recharge
3,2020-11-01,26.26,19321,send_money
4,2020-11-01,35.35,38438,send_money
...,...,...,...,...
756478,2020-11-30,17.76,74524,send_money
756479,2020-11-30,111.02,42594,transport
756480,2020-11-30,0.81,1338,transport
756481,2020-11-30,34.71,1338,credits_consumer


In [291]:
filtered_df_last_week_users_prop_pays = pd.merge(df_pays, df_last_week, how='inner', left_on=['user_id_pays', 'value_prop_pays'], right_on=['user_id_prints_last_week', 'value_prop_prints_last_week'])
df_last_three_weeks_pays = filtered_df_last_week_users_prop_pays[(filtered_df_last_week_users_prop_pays['pay_date_pays'] >= three_weeks_ago) & (filtered_df_last_week_users_prop_pays['pay_date_pays'] <= now)]
counts_pays = df_last_three_weeks_pays.groupby(['user_id_pays', 'value_prop_pays']).size()
df_grouped_total_pays = df_last_three_weeks_pays.groupby(['user_id_pays', 'value_prop_pays'])['total_pays'].sum().reset_index()
counts_df_pays = counts_pays.reset_index(name='counts_pays')

,user_id_pays,value_prop_pays,counts_pays
0,1,cellphone_recharge,2
1,1,link_cobro,1
2,3,link_cobro,1
3,3,prepaid,3
4,6,prepaid,1
...,...,...,...
65666,99994,send_money,1
65667,99996,credits_consumer,1
65668,99997,credits_consumer,1
65669,100000,prepaid,1


In [294]:
merged_q4 = pd.merge(merged_q3, counts_df_pays, how='left', left_on=['user_id_prints_last_week', 'value_prop_prints_last_week'], right_on=['user_id_pays', 'value_prop_pays'])
merged_q4['count_number_payments_3_weeks'] = merged_q4['counts_pays'].fillna(0)

In [298]:
merged_q5 = pd.merge(merged_q4, df_grouped_total_pays, how='left', left_on=['user_id_prints_last_week', 'value_prop_prints_last_week'], right_on=['user_id_pays', 'value_prop_pays'])
merged_q5['total_pays'] = merged_q5['total_pays'].fillna(0)

In [299]:
result = merged_q5[['day_prints_last_week','user_id_prints_last_week','position_prints_last_week','value_prop_prints_last_week','clicked_or_not','count_number_prop_views_3_weeks', 'count_number_prop_click_3_weeks', 'count_number_payments_3_weeks', 'total_pays']]

In [301]:
result

,day_prints_last_week,user_id_prints_last_week,position_prints_last_week,value_prop_prints_last_week,clicked_or_not,count_number_prop_views_3_weeks,count_number_prop_click_3_weeks,count_number_payments_3_weeks,total_pays
0,2020-11-23,69000,0,credits_consumer,False,1,0.0,2.0,122.19
1,2020-11-23,69000,1,link_cobro,False,1,0.0,1.0,152.51
2,2020-11-23,69000,2,transport,False,2,0.0,0.0,0.00
3,2020-11-23,69000,3,prepaid,False,1,0.0,1.0,106.65
4,2020-11-23,66521,0,prepaid,False,4,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...
127549,2020-11-30,50807,0,send_money,False,1,0.0,1.0,107.31
127550,2020-11-30,50807,1,prepaid,False,2,0.0,0.0,0.00
127551,2020-11-30,50807,2,credits_consumer,False,3,0.0,2.0,87.07
127552,2020-11-30,1487,0,point,False,1,0.0,2.0,14.85
